# Quantum Harmonic Oscillator

We want to solve the quantum harmonic oscillator numerically. The QHO has the potential energy
$$
V(x) = x^2
$$

## Auxiliary code

This is some code that we will use further down to run our calculations.

We start with the usual imports.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

### The Implementation of the Numerov Method

We are using the Numerov method that we introduce for the particle in a box.

In [ ]:
def numerov(x,q,s):

    u = np.zeros(len(q))
    u[0] = 0
    u[1] = x[1]-x[0]

    for i in range(1,len(u)-1):
        g = (x[i]-x[i-1])**2 / 12.
        c0 = 1 + g * q[i-1]
        c1 = 2 - 10 * g * q[i]
        c2 = 1 + g * q[i+1]
        d  = g * (s[i+1] + s[i-1] + 10*s[i])
        u[i+1] = ( c1*u[i] - c0*u[i-1] + d ) / c2

    return u

### The Trapezoid Integration Method

We use this to normalize the wave function, but it could be implemented more elegantly.

In [ ]:
def trapezoid(x,y):
    sum = 0.0
    for i in range(0,len(y)-1):
        sum += 0.5 * ( y[i] + y[i+1] ) * (x[i+1] - x[i])
    return sum

# Demonstration: How to Match the Integrated Functions

We start by defining our problem: we set the x-domain and define our potential on that domain. We also define the functions $q$ and $s$ required for the Numerov method.

In [ ]:
x = np.linspace(-10,10,500)
V = 0.5*x*x
E = 0.6

s = np.zeros(len(V))
q = 2*(E - V)

If we try to integrate over the entire domain, we observe exponential growth.

In [ ]:
plt.plot(x,numerov(x,q,s))

We can work around this by integrating from the left and the right hand side, and plot the functions that we find.

In [ ]:
plt.plot(x[:280],numerov(x[:280],q[:280],s[:280]))
plt.plot(x[-1:-280:-1],numerov(x[-1:-280:-1],q[-1:-280:-1],s[-1:-280:-1]));

In general, the two functions will not match. Rather than fulfilling the boundary condition on the "far side", we now have to match the functions that we integrated from the left and the right hand side, i.e. we have to equate the function value and first derivative at one point.

For numerical stability, it is best do do the matching where the first derivate is maximal, i.e. where the second derivative is zero, or where we have a turning point of the function. 

In [ ]:
# The matching is best done at the turning point, so we need to find it. 
# At the turning point, we expect a sign change of the q:
turnpoints = np.argwhere(q[:-1]*q[1:]<0)

if len(turnpoints)>0:
    # lets use the last turning point
    im = turnpoints[-1][0]
else:
    print ("turning point not found for energy E=%f" % E)
    # let's use some thing just off the middle
    im = int(0.6*len(q))

# Based on this, we can slice the domain into two parts. The left slice
# runs from the beginning to one beyond the turning point, and the right
# slice runs from the end back to overlap with the left slice at two 
# points.
sl = slice(0,im+2)
sr = slice(-1,-(len(q)-im+1),-1)

# And we can check that we slice at the right place
for i in (sl,sr):
    print("Slice:",i)
    print("Last values of x", x[sl][-2:])
    print("Last values of q", q[sl][-2:])

In [ ]:
# Now we integrate from the left and from the right with the Numerov 
# method
ul = numerov(x[sl],q[sl],s[sl])
ur = numerov(x[sr],q[sr],s[sr])

# And we can take a look at the last values
print("Last values of ul", ul[-2:])
print("Last values of ur", ur[-2:])

# We now have two points of overlap between the two functions:
#   at x[im]:   ur[-2]   and    ul[-1]
#   at x[im+1]  ur[-1]   and    ul[-2]

# We match ur and ul at the last data point of ul, at x[im+1]:
ur = ul[-1] / ur[-2] * ur
print("Rescaled last values of ur", ur[-2:])


plt.plot(x[im-5:im+2],ul[-7:])
plt.plot(x[im:im+7],ur[-1:-8:-1])

In [ ]:
# We use the value at x[im] as the matching condition
#
# This is somewhat simpler than what T. Pang did, but we just need
# to check if the derivatives of ul and ur match. Because we have
# already rescaled ur, we can go directly into the comparison.
f = ul[-2] - ur[-1]
print(f)

In [ ]:
# Now we can concatenate the two parts into a single wavefunction
u = np.concatenate( [ul[0:-1], ur[-2::-1]] )

#     # We calculate the normalization before we return it.
#     norm = np.sqrt(trapezoid(x,u*u))

#     # And finally we return the normalized matching condition and wave
#     # function
#     return f/norm, u/norm

plt.plot(x,u)

In [ ]:
def secantsearch(e1,e2,x,V):

    f1,u = wave(x,e1,V)
    f2,u = wave(x,e2,V)


    # run secant search
    while np.abs(f2) > 1E-10:

        # the actual secant method
        e3 = (e1*f2 - e2*f1) / (f2-f1)

        # check for pathetic values
        #if e3<0:
        #    e3 = 0.1

        if e3==e2:
            e3 = 0.9 * e2 + 0.1 * e1

        # update choice
        e1 = e2
        f1 = f2

        e2 = e3
        f2,u = wave(x,e2,V)

        #print ("  trial: E=%5f f(E)=%f" % (e2,f2))


    # secant method converged, we found the eigenvalue
    #f,u = wave(x,e2,V)


    return e2,u

In [ ]:
def wave(x,E,V):
    s = np.zeros(len(V))
    q = 2*(E - V)

    # Find the matching point at the right turning point.  'im' will
    # be the index of the last point before E-V changes sign.
    im = -1
    for i in range(len(q)-1):
        if q[i]*q[i+1]<0:
            "  sign change u[{}] = {} u[{}] = {}".format(
                i, q[i], i+1, q[i+1])

        if q[i]*q[i+1]<0 and q[i]>0:
            im = i

    if im < 0:
        print ("turning point not found for energy E=%f" % E)
        im = int(0.6*len(q))
        #return


    # First, integrate u from the left, up to the point just right of
    # the turning point, i.e. up to im+1. We need in total im+2 data
    # points, because the point numbering starts at 0.
    nl = im+2
    ul = numerov(x[0:nl], q[0:nl],s[0:nl])

    # We then integrate u from the right. We want two points of
    # overlap with ul. Note that the order of ur is reversed, i.e. the
    # leftmost point has the highest index.
    nr = len(q) - nl + 2
    ur = numerov(x[-1:-nr-1:-1], q[-1:-nr-1:-1], s[-1:-nr-1:-1])


    # We now have two points of overlap between the two functions:
    #   at x[im]:   ur[-2]   and    ul[-1]
    #   at x[im+1]  ur[-1]   and    ul[-2]

    # We match ur and ul at the last data point of ul, at x[im+1]:
    ur = ul[-1] / ur[-2] * ur

    # We use the value at x[im] as the matching condition
    #
    # This is somewhat simpler than what T. Pang did, but we just need
    # to check if the derivatives of ul and ur match. Because we have
    # already rescaled ur, we can go directly into the comparison.
    f = ul[-2] - ur[-1]

    # Now we can concatenate the two parts into a single wavefunction
    u = np.concatenate( [ul[0:-1], ur[-2::-1]] )

    # We calculate the normalization before we return it.
    norm = np.sqrt(trapezoid(x,u*u))

    # And finally we return the normalized matching condition and wave
    # function
    return f/norm, u/norm


In [ ]:
def draw_wave(x,E,V):
    f,u = wave(x,E,V)
    print ("mismatch: %f" % f)
    plt.plot(x,u)
    plt.show()

In [ ]:
x    = np.linspace(-10,10,500)
Vqho = 0.5*x*x

draw_wave(x,0.2,Vqho)

In [ ]:
x    = np.linspace(-10,10,500)
Vqho = 0.5*x*x
Vpot = x**2
# Vpot = - 50. / np.cosh(x)**2


plt.xlim(-4,4)
#plt.ylim(-1,8)
# plt.plot(x,Vpot)

e,u = secantsearch(0.1, 0.3,x,Vpot)

for e1 in np.linspace(0.4, 7.1,8):
    e,u = secantsearch(e1,1.1*e1,x,Vqho)

    print ("Eigenvalue: %f" % e)

    plt.plot(x,u*u + e)


if 'PLOT_AUTOSAVE_DIR' in os.environ:
    plt.savefig("%s/qho.pdf" % os.environ['PLOT_AUTOSAVE_DIR'])
else:
    plt.show()